## Berechnungen von Merkmalen

In diesem Notebook ist es möglich den Mittelwert, Varianz und Standardabweichung für die aufgenommenen Messwerte aus der Datei "merkmaleRoh.csv" der Android TM Fahrradwege Monitoring App zu berechnen. Zu einem Bild werden parallel Messwerte erfasst von dem Beschleunigungssensor und Magnetsensor des Smartphones, innerhalb eines Zeitfenster mit einer Länge von 80ms (siehe hierzu auch das Notebook "gatheringSequence" in dem Ordner datenvisualisierung). Bei dem Beschleunigungssensor sind dies die Messwerte der beschleunigung von den 3-Achsen X, Y sowie Z (Abb. 1). Des Weiteren werden die Gier-Nick-Roll Winkel in rad in einem Zeitfenster erfasst. Für die Berechnung des Mittelwerts, Varianz und Standardabweichung über die Messwerte wird die Form der abgespeicherten Messwerte geändert, weil diese als Python Objekte verwaltet werden mit welchen nicht gerechnet werden kann. Zu den Messwertreihen wird jeweils eine CSV Datei erstellt. Eine Reihe stellt dann in dieser Datei eine Messreihe zu einem Zeitfenster dar. Bei der ausführung dieses Notebooks werden deshalb zusätzlich folgende CSV Dateien erstellt:

X-Achse Beschleunigungswerte in m/s^2   
Y-Achse Beschleunigungswerte in m/s^2  
Z-Achse Beschleunigungswerte in m/s^2   
Gier Messwerte in rad  
Nick Messwerte in rad  
Roll Messwerte in rad   
Zeitstempel der Messwerte in ns  

<img src="../daten/abbildungen/sensorOrientierung.png" alt="Android Smartphone Sensor Orientierung" width="200px" />
Abbildung 1: 3-Achsen Smartphone Sensor-Koordinatensystem   
Quelle: siehe Bildverzeichnis Abb. 2 in der README.md Datei

In [6]:
import pandas
import numpy as np
import numpy.testing as npt
import math
import os

In [7]:
dataset = '39'

In [8]:
featuresDf = pandas.read_csv("../daten/merkmale_datensatz_" + dataset + "/merkmaleRoh.csv")
featuresDf.describe()

,Zeitstempel in Unixzeit,Breitengrad,Laengengrad,Geschwindigkeit in km/h,Anzahl der Messwerte,Start des Zeitfensters in ns seit Start der JVM,Zeitstempel Messwertdaten anfordern in Unixzeit,Start der Messwerterfassung in ns seit Start der JVM,Erster Zeitstempel der Teilliste in ns seit Start der JVM,Start der Belichtung in ns seit Start der JVM,Belichtungszeit in ns,Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM,Speicherzeitpunkt der Merkmale in Unixzeit
count,1.757100e+04,17571.000000,17571.000000,17571.000000,17571.000000,1.757100e+04,1.757100e+04,1.757100e+04,1.757100e+04,1.757100e+04,1.757100e+04,1.757100e+04,1.757100e+04
mean,1.528125e+12,53.582725,10.085151,15.692816,29.906494,3.620444e+13,1.528125e+12,3.616923e+13,3.620377e+13,3.620455e+13,1.678901e+07,3.620469e+13,1.528125e+12
std,2.628825e+06,0.016063,0.039181,4.084292,11.115786,2.623375e+12,2.628821e+06,2.631319e+12,2.623417e+12,2.623370e+12,8.957916e+06,2.623364e+12,2.628820e+06
min,1.528121e+12,53.556300,10.013530,5.004000,15.000000,3.176904e+13,1.528121e+12,3.176885e+13,3.176885e+13,3.176934e+13,3.171539e+06,3.176966e+13,1.528121e+12
25%,1.528123e+12,53.570538,10.051223,13.104001,23.000000,3.405012e+13,1.528123e+12,3.398748e+13,3.404961e+13,3.405022e+13,1.000184e+07,3.405034e+13,1.528123e+12
50%,1.528125e+12,53.578903,10.092756,15.984000,26.000000,3.604473e+13,1.528125e+12,3.594348e+13,3.597039e+13,3.604489e+13,1.000184e+07,3.604507e+13,1.528125e+12
75%,1.528128e+12,53.595013,10.122481,18.612000,33.000000,3.864334e+13,1.528128e+12,3.861582e+13,3.864287e+13,3.864346e+13,2.999632e+07,3.864358e+13,1.528128e+12
max,1.528129e+12,53.613860,10.137978,27.684000,89.000000,4.022620e+13,1.528129e+12,4.020058e+13,4.022565e+13,4.022647e+13,3.287369e+07,4.022679e+13,1.528129e+12


In [9]:
featuresDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17571 entries, 0 to 17570
Data columns (total 17 columns):
Zeitstempel in Unixzeit                                               17571 non-null int64
Breitengrad                                                           17571 non-null float64
Laengengrad                                                           17571 non-null float64
Geschwindigkeit in km/h                                               17571 non-null float64
Z-Achse Beschleunigungswerte in m/s^2                                 17571 non-null object
Y-Achse Beschleunigungswerte in m/s^2                                 17571 non-null object
Nick Messwerte in rad                                                 17571 non-null object
Zeitstempel der Messwerte in ns                                       17571 non-null object
Anzahl der Messwerte                                                  17571 non-null int64
Start des Zeitfensters in ns seit Start der JVM               

In [ ]:
# Zeigt die ersten 5 Reihen Beschleunigungssensordaten der X-Achse 
featuresDf['X-Achse Beschleunigungswerte in m/s^2'].head()

In [ ]:
# Die Anzahl der Messwerte sollte immer gleich sein, ansonsten werden fehlende Spalten mit NaN Werten gefüllt bei der
# späteren Abspeicherung der erfassten Messwerte wie z.B. in die Datei x-Achse-Beschleunigungswerte.csv.
# Um die Berechnung von den Merkmalen Varianz und Mittelwert unteranderem zu beschleunigen ist es wichtig sogenannte Außenseiter
# aus den Datensätzen zu entfernen.
# Dafür muss auch das Bild zu einer Reihe aus der merkmaleRoh.csv Datei gelöscht werden.
# Dies wird erreicht indem alle Pfade der Bilder geladen werden.
# Dann können Reihen sowie Bilder gelöscht werden mit mehr Messwerten als ein bestimmter Grenzwert.
print(featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > 23])

################### Entferne Außenseiter mit Anzahl der Messwerte > Grenzwert ###################
#### Warnung löschen der zugeschnittennen Bilder kann nicht rückgängig gemacht werden!
#threshold  = 23 
#imagePaths = []
# Hier können die Datensätze ausgewählt werden
#datasets = ['15','16','17','18','19','20','21','22']
# Die Pfade zu den Ordnern in welchem sich die Bilder befinden
#paths = []
#for dataset in datasets: # Für jeden Datensatz merke Pfad
#    paths.append("C:/Users/morro/Documents/datenRoh/" + dataset + "/zugeschnitten/")
#for path in paths: # Für jeden Pfad hole die Namen der Ordner
#    folders = os.listdir(path)
#    folders = sorted(folders, key=int) #sortiert die Reihenfolge de Ordner aufsteifend
#    print(folders)
#    for folder in folders: # Aus der Liste der Ordner wird ein Ordner ausgewählt
#        filesPath = path + folder + "/"
#        files = os.listdir(filesPath)
#        for name in files: # Ein Dateiname aus diesem Ordner
#            if "jpg" not in name:
#                continue
#            imagePaths.append(filesPath + name) # Path wird geladen in Liste
#toRemoveImagesIndex = featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > threshold].index.tolist()
#for i in toRemoveImagesIndex:
#    print(imagePaths[i])
#    os.remove(imagePaths[i])
unwantedRows = featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > 23]
featuresDf.drop(unwantedRows.index, inplace=True)
# Die Inecies müssen zurückgesetzt werden nach den Aufruf von drop. Ansonsten kommte es zu KeyValue Fehlern
featuresDf.reset_index(drop=True,inplace=True)

In [ ]:
featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > 23]

Die folgenden Spalten des Datenframes haben als Type ein Python object.   
X-Achse Beschleunigungswerte in m/s^2   
Y-Achse Beschleunigungswerte in m/s^2   
Z-Achse Beschleunigungswerte in m/s^2   
Gier Messwerte in rad    
Nick Messwerte in rad          
Roll Messwerte in rad                   
Zeitstempel der Messwerte in ns               
Implizit haben diese Spalten als Typ Strings. Aber in einem DataFrame werden Strings als    
Python objects erkannt. Um den Datentyp auf float zu casten wird aus den Spalten zunächst ein Liste erstellt.    
Welche dann wiederum als Numpy Array auf den Typ float konvertiert wird.   

In [ ]:
featuresDf.info()

In [ ]:
accXList = featuresDf['X-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()

In [ ]:
# Direkt mit dem Numpy Array ist es nicht möglich die Konvertierung durchzuführen, weil die Arrays von Typ object sind.   
pandas.DataFrame(featuresDf['X-Achse Beschleunigungswerte in m/s^2'].str.split(" ")).values

In [ ]:
accXList[12] # zeige Liste

In [ ]:
np.array(accXList[1]).dtype

In [ ]:
# Konvertiert die Python Objekte in der Liste zu den Type Float.
# Das Ergebnis wird in einem Dataframe geschrieben
accList = []
accList.append(pandas.DataFrame(abs(np.array(accXList[0]).astype(float))).transpose())
for x in accXList[1:]:
    accList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())

In [ ]:
accXDf = pandas.concat(accList, ignore_index=True)

In [ ]:
accXDf.head()

In [ ]:
# Um mehr Spalten eines Panda DataFrames auszugeben
pandas.set_option('display.max_columns', 40)
accXDf.T.mean().head(20)

In [ ]:
accXDf.shape

In [10]:
accYList = featuresDf['Y-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(accYList[0]).astype(float)).transpose())
for x in accYList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accYDf = pandas.concat(accList, ignore_index=True)

In [ ]:
accYDf.info()

In [ ]:
accYDf.head()

In [11]:
# Konvertiere Beschleunigungssensordaten Z-Achse zu einem DataFrame
measuredDataList = featuresDf['Z-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accZDf = pandas.concat(accList, ignore_index=True)

In [12]:
# Konvertiere Nick Messwerte zu DataFrame
measuredDataList = featuresDf['Nick Messwerte in rad'].str.split(" ").tolist()
pitchList = []
pitchList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    pitchList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
nickDf = pandas.concat(pitchList, ignore_index=True)

In [ ]:
# Konvertiere Roll Messwerte zu DataFrame
measuredDataList = featuresDf['Roll Messwerte in rad'].str.split(" ").tolist()
rollList = []
rollList.append(pandas.DataFrame(abs(np.array(measuredDataList[0]).astype(float))).transpose())
for x in measuredDataList[1:]:
    rollList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())
rollDf = pandas.concat(rollList, ignore_index=True)

In [ ]:
measuredDataList = featuresDf['Gier Messwerte in rad'].str.split(" ").tolist()
azimuthList = []
azimuthList.append(pandas.DataFrame(abs(np.array(measuredDataList[0]).astype(float))).transpose())
for x in measuredDataList[1:]:
    azimuthList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())
azimuthDf = pandas.concat(azimuthList, ignore_index=True)

In [13]:
measuredDataList = featuresDf['Zeitstempel der Messwerte in ns'].str.split(" ").tolist()
timestampsList = []
timestampsList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    timestampsList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
sensorTimestampsDf = pandas.concat(timestampsList, ignore_index=True)

In [14]:
# Neue Spalten werden erstellt und Mittelwerte zugewiesen
#featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'] = accXDf.T.mean()
featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'] = accYDf.T.mean()
featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'] = accZDf.T.mean()
featuresDf['Mittelwert Nick in rad'] = nickDf.T.mean()
#featuresDf['Mittelwert Roll in rad'] = rollDf.T.mean()

In [ ]:
#featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'].head(2)

In [ ]:
featuresDf.columns

Berechnet die Varianz. Dieser Funktion muss als Paramter der Mittelwert (mean) und der DataFrame mit   
den Float Werten übergeben werden, um die Varianz zu berechnen. Als Varianz wird der Durchschnitt der quadrierten  
Differenzen zum Mittelwert bezeichnet.   
Prec.:    
Postc.: Gibt die berechnete Variance als Float zurück oder 0 wenn die Anzahl der Werte <= 0  

In [15]:
def calcVariance(meansDf, dfValues):
    variance = []
    for i, values in dfValues.iterrows():
        sum = 0
        for value in values:
            if math.isnan(value) == False:
                tempDifference = value-meansDf[i]
                sum += tempDifference * tempDifference
        variance.append("{0:.5f}".format(round(sum / values.count(),5)))
    return np.array(variance).astype(float)

Hier wird die Standardabweichung der Varianz berechnet. Dies ist die Wurzel der Varianz.   
Für die Berechnung wird der Absolutwert von der Varianz genommen.    
Prec.:
Postc.:  Standardabweichung wird zurückgegeben.

In [16]:
def calculateStandardDeviation(varianceDf):
    deviation = []
    for v in varianceDf:
        temp = math.sqrt(np.abs(v))
        deviation.append("{0:.5f}".format(round(temp,5)))
    return np.array(deviation).astype(float)

Die Funktion berechnet die Winkeldifferenz in rad zwischen den ersten gemessenen Gierwinkel und den letzten Gierwinkel   
innerhalb einer als Argument übergebenen DataFrames. Der zurückgegebene Radiant    
ist immer positiv und gibt die relative änderung in rad an.      
Prec.:   
Postc.: Radiant berechnet   

In [17]:
def calculateAngelChangeAzimuth(azimuthDf):
    result = []
    for i,values in azimuthDf.iterrows():
        first = values[0]*(180/math.pi)
        last = values.iloc[values.count()-1]*(180/math.pi)
        resultTemp = np.abs(first - last)
        if(resultTemp > 180):
            resultTemp = 360 - resultTemp
        result.append("{0:.5f}".format(round(resultTemp * ( math.pi / 180),5)))
    return np.array(result).astype(float)

In [ ]:
featuresDf['Gier-Aenderung in rad'] = calculateAngelChangeAzimuth(azimuthDf)

In [ ]:
accXDf[14957:14961]

In [ ]:
featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'].iloc[10]

In [18]:
# Berechnung der Varianzen und hinzufügen der Ergebnisse als neue Spalte in dem DataFrame 
#featuresDf['Varianz X-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'],accXDf)
featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'],accYDf)
featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'],accZDf)
featuresDf['Varianz Nick in rad'] = calcVariance(featuresDf['Mittelwert Nick in rad'],nickDf)
#featuresDf['Varianz Roll in rad'] = calcVariance(featuresDf['Mittelwert Roll in rad'],rollDf)

In [19]:
# Berechne Standardabweichung
#featuresDf['Standardabweichung X-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz X-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Y-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Z-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Nick in rad'] = calculateStandardDeviation(featuresDf['Varianz Nick in rad'])
#featuresDf['Standardabweichung Roll in rad'] = calculateStandardDeviation(featuresDf['Varianz Roll in rad'])

In [20]:
# Änderungen in CSV Datei schreiben
featuresDf.set_index('Zeitstempel in Unixzeit', inplace=True)
featuresDf.to_csv('../daten/merkmale_datensatz_'+ dataset + '/merkmale.csv')

In [21]:
featuresDf = pandas.read_csv("../daten/merkmale_datensatz_" + dataset + "/merkmale.csv")

In [22]:
# Speichert die DatenFrames mit den Messwerten in csv Dateien
#accXDf.index.names = ["index"]
#accXDf.to_csv('../daten/merkmale_datensatz_26_bis_27/absMesswerte/xAchseBeschleunigungswerte.csv')
accYDf.index.names = ["index"]
accYDf.to_csv('../daten/merkmale_datensatz_' + dataset + '/yAchseBeschleunigungswerte.csv')
accZDf.index.names = ["index"]
accZDf.to_csv('../daten/merkmale_datensatz_' + dataset + '/zAchseBeschleunigungswerte.csv')
nickDf.index.names = ["index"]
nickDf.to_csv('../daten/merkmale_datensatz_' + dataset + '/nickMesswerte.csv')
#rollDf.index.names = ["index"]
#rollDf.to_csv('../daten/merkmale_datensatz_26_bis_27/absMesswerte/rollMesswerte.csv')
#azimuthDf.index.names = ["index"]
#azimuthDf.to_csv('../daten/merkmale_datensatz_26_bis_27/absMesswerte/azimuthMesswerte.csv')
sensorTimestampsDf.index.names = ["index"]
sensorTimestampsDf.to_csv('../daten/merkmale_datensatz_' + dataset + '/sensoreventZeitstempel.csv')

In [ ]:
# Unittests
import unittest

class CalcTest(unittest.TestCase):
    
    
    # Testet die Funktion calcVariance mit einem DataFrame mit den Werten [2.0, 2.0]
    # Der Mittelwert sollte 2.0 sein
    # Das erwartete Ergebnis ist 0.0
    def testCalcVariance2(self):
        meanL = [2.0]
        meanDf = pandas.DataFrame(meanL)
        valuesA = [2.0,2.0]
        valuesDf = pandas.DataFrame([[2.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 0.0,2)
        
    # Testet die Funktion calcVariance mit einem DataFrame mit den folgenden Werten:
    # [1.24, 2.5213, 10.434, 42.45, 5.9]
    # Der Mittelwert ist 12.508
    # Die erwartete Varianz ist 234,247016
    def testCalcVariance5(self):
        meanDf = pandas.DataFrame([12.508])
        valuesDf = pandas.DataFrame([[1.24, 2.52, 10.43, 42.45, 5.9]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 234.24702,5)   
    
    # Testet calcVariance mit 0 Datenwerten   
    def testCalcVariance0(self):
        meanDf = pandas.DataFrame([0.0])
        valuesDf = pandas.DataFrame([[0.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),0.0,2)  
        
    # Testet calcVariance mit negativen Datenwerten   
    def testCalcVarianceNegative(self):
        meanDf = pandas.DataFrame([-24.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),451.5,1) 
        
    # Testet calcVariance mit Muultidimensionalen DataFrame   
    def testCalcVarianceMultidim(self):
        meanDf = pandas.DataFrame([-24.0,0.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0],[0.0,0.0,0.0,0.0]])
        # Hier wird Numpy Test verwendet um das Ergebnis (zwei Arrays) zu vergleichen
        npt.assert_array_equal(calcVariance(meanDf[0],valuesDf),np.array([ 451.5,0.0])) 
        
    # Dieser Test testet die Funktion calculateStandardDeviation.
    def testCalculateStandardDeviation(self):
        varianceDf = pandas.DataFrame([451.5])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),21.24853,5) 
        
    # Testet die Funktion calculateStandardDeviation.
    # Dabei hat das Argument den Wert 0.0.
    def testCalculateStandardDeviationZero(self):
        varianceDf = pandas.DataFrame([0.0])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),0.0,2)
    
    #  Testet die Funktion calculateAngelChangeAzimuth mit zwei gleichen rad Werten
    def testCalculateAngelChangeAzimuthEqual(self):
        radiants = pandas.DataFrame([[1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),0.0,2)
    
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten -1.0 und 1.0
    def testCalculateAngelChangeAzimuthOne(self):
        radiants = pandas.DataFrame([[-1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 1.0 und -1.0
    def testCalculateAngelChangeAzimuthOne2(self):
        radiants = pandas.DataFrame([[1.0,-1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  

    # Testet die Funktion calculateAngelChangeAzimuthValues mit mehreren Werten
    def testCalculateAngelChangeAzimuthValues(self):
        radiants = pandas.DataFrame([[-1.57,1.0,2.0,1.3,1.57]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),3.14,2)
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 0.05 und 0.06
    def testCalculateAngelChangeAzimuthSmall(self):
        radiants = pandas.DataFrame([[0.01,0.02]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 0.01,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde mit 
    # negative Radianten 
    def testCalculateAngelChangeAzimuthNegativ(self):
        radiants = pandas.DataFrame([[-2.9,-2.0,-1.28]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 1.62,2) 

unittest.main(argv=[''], verbosity=2, exit=False)

In [ ]:
featuresDf['Standardabweichung Z-Achse Beschleunigung in m/s^2'].iloc[27906-30]

In [ ]:
featuresDf.iloc[27906-31]

In [ ]:
featuresDf[featuresDf['Standardabweichung Z-Achse Beschleunigung in m/s^2']]

In [ ]:
featuresDf[featuresDf['Anzahl der Messwerte'] == featuresDf['Anzahl der Messwerte'].max()]